# Integrating Multiple Agentic Frameworks
In this tutorial, we explore how to combine LangGraph, a flexible framework for building agentic workflows, with AutoGen, a powerful agent system from Microsoft, to build an intelligent chatbot that:

* Supports persistent memory between interactions
* Uses AutoGen's reasoning abilities to handle complex tasks
* Leverages LangGraph's features like task routing, entrypoints, and streaming output

---
Why integrate LangGraph with other frameworks?
LangGraph provides strong abstractions for stateful, multi-step applications. Integrating external agent frameworks like AutoGen enables:


*   Modular agent design across ecosystems
*   Seamless use of LangGraph features (memory, streaming, persistence)
*   Support for hybrid multi-agent workflows

---

### Step 1: Install Dependencies

In [ ]:
%pip install autogen-agentchat[gemini]~=0.2 langgraph litellm

### Step 2: Configure OpenAI API Key

In [ ]:
import getpass
from google.colab import userdata
import os
import litellm # Imported as an architecture hint although not used directly. Autogen uses it under the hood.

os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

OAI_CONFIG_LIST = [
    {
        "model": "gpt-35-turbo",
        "api_key": "your OpenAI Key goes here",
    },
    {
        "model": "gemini-2.0-flash",
        "api_key": os.environ.get("GEMINI_API_KEY"),
        "api_type": "google"
    }
]

### Step 3: Define AutoGen Agents

We’ll use AutoGen's AssistantAgent and UserProxyAgent. These agents can auto-respond and run code when needed.

In [ ]:
import autogen
import os


config_list_gemini  = [
    {
        "model": "gemini-2.5-flash",
        "api_key": os.environ.get("GEMINI_API_KEY"),
        "api_type": "google"
    }
]

llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list_gemini,
    "temperature": 0,
}

autogen_agent = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "web",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    llm_config=llm_config,
    system_message="Reply TERMINATE if the task has been solved at full satisfaction. Otherwise, reply CONTINUE, or the reason why the task is not solved yet.",
)

### Step 4: Create the LangGraph Workflow

Let’s build a LangGraph graph that delegates reasoning to AutoGen, with support for short-term memory and checkpointing.

In [ ]:
from langchain_core.messages import convert_to_openai_messages, BaseMessage
from langgraph.func import entrypoint, task
from langgraph.graph import add_messages
from langgraph.checkpoint.memory import MemorySaver


@task
def call_autogen_agent(messages: list[BaseMessage]):
    # convert to openai-style messages
    messages = convert_to_openai_messages(messages)
    response = user_proxy.initiate_chat(
        autogen_agent,
        message=messages[-1],
        # pass previous message history as context
        carryover=messages[:-1],
    )
    # get the final response from the agent
    content = response.chat_history[-1]["content"]
    return {"role": "assistant", "content": content}


# add short-term memory for storing conversation history
checkpointer = MemorySaver()


@entrypoint(checkpointer=checkpointer)
def workflow(messages: list[BaseMessage], previous: list[BaseMessage]):
    messages = add_messages(previous or [], messages)
    response = call_autogen_agent(messages).result()
    return entrypoint.final(value=response, save=add_messages(messages, response))

###  Step 5: Run the Graph

Initialize a Config with Thread ID

In [ ]:
# pass the thread ID to persist agent outputs for future interactions
# highlight-next-line
config = {"configurable": {"thread_id": "2"}}

for chunk in workflow.stream(
    [
        {
            "role": "user",
            "content": "Which numbers between 1 and 50 are divisible by 7?",
        }
    ],
    # highlight-next-line
    config,
):
    print(chunk)

### **Step 6: Continue the Conversation**
Because we're using LangGraph’s persistence layer, we can pick up the thread in future runs:


In [ ]:
for chunk in workflow.stream(
    [
        {
            "role": "user",
            "content": "Multiply the last number by 3",
        }
    ],
    # highlight-next-line
    config,
):
    print(chunk)